### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import shap
pd.set_option('display.max_columns', None)

/Users/leandrog/opt/anaconda3/envs/ml_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Preprocessing Data
We start preprocessing the geochemical and magnetic susceptibility datasets
for further analysis. 

The following steps include loading the susceptibility and geochemistriy datasets, merge on the sample 
and cleaning the data.

We first get the raw data and resolve `NaN` values. In this first approach we get rid of the columns with `NaN`s.

In [3]:
# This dataset contains geochemical measurements for different samples.
df_xrf_corrected = pd.read_csv('../Data/TG_xrf_corrected.csv')

# This dataset includes magnetic susceptibility measurements for the samples.
df_susc = pd.read_csv('../Data/TG_Bulk_Susceptibility_all.csv')

# We perform a left join to retain all XRF data and add susceptibility values where available.
df_all = pd.merge(df_xrf_corrected, df_susc[['sample','X_fv']], on='sample', how='left')

### Remove samples with missing susceptibility values
df_all = df_all[~df_all['X_fv'].isna()]

### Drop categorical identifiers
# The 'sample' and 'section' columns are metadata and are not needed for numerical analysis.
df_all = df_all.drop(columns = ['sample', 'section'])

### Handling Missing Data
Before performing further analysis, we check for `Nan` values in the dataframe (<LOD; below limit of detection). If any missing values exist, we replace them with 0.0

In [ ]:
nan_counts = df_all.isna().sum()
print("Number of nans: ", np.sum(nan_counts))

# Fill NaNs with 0.0 and verify that no missing values remain
df = df_all.fillna(0.0)
assert np.sum(df.isna().sum()) == 0

### Compute Ratios


In [ ]:
add_ratios = True

if add_ratios:
    
    df['Rb/Sr'] = df['Rb'] /  df['Sr']
    df['Ca/Al'] = df['Ca'] /  df['Al2O3']
    df['Fe/S']  = df['Fe'] / df['S']
    df['Si/Al'] = df['SiO2'] / df['Al2O3']
    df['K/Al']  = df['K2O'] / df['Al2O3']
    df['K/Rb']  = df['K2O'] / df['Rb']
    df['Rb/Al'] = df['Rb'] / df['Al2O3']
    df['Si/Al'] = df['SiO2'] / df['Ca']
    df['Ti/Al'] = df['Ti'] / df['Al2O3'] # https://cp.copernicus.org/articles/20/415/2024/cp-20-415-2024.pdf

    # Replace nans again but now by infinity
    print("Total number of NaNs after ratios: ", np.sum(df.isna().sum()))
    df = df.replace([np.inf, -np.inf], 999)

df.head(5)

We now separate the feature matrix ($X$) from the target data ($Y$):

In [ ]:
target = 'X_fv' 

# Feature matrix
X = df.drop(columns=[target])  
feature_names = X.columns

# Response vector
Y = df[target] 

X.shape, Y.shape

We scale the data to have zero mean and unit variance. This is not required for many ML methods, including random forrest and XGBoost, but it does not hurt to do it and we have seen this leads to better result performances. See the [issue](https://github.com/scikit-learn/scikit-learn/issues/29922#issuecomment-2460276129) in the Scikit-learn documentation for more information. 

In [ ]:
def normalize(Z, mode):
    # Standard scaler to have zero mean and unit variance
    norm = StandardScaler().fit(Z)    
    # transform your training data
    return norm.transform(Z)

X_non_norm = X
Y_non_norm = Y

# X = normalize(X)
# Y = normalize(np.array(Y).reshape(-1, 1)).reshape(-1)

X = X
Y *= 10 ** 10 

# Random Forest Model for Regression

In this cell, we initialize and train a `RandomForestRegressor` model with 300 trees (`n_estimators=300`) and a maximum depth of 4 (`max_depth=4`).  
We fit the model on `X` (features) and `Y` (target variable), then make predictions on `X`.  
Finally, we evaluate the model's performance using Root Mean Squared Error (RMSE) and R² Score.


In [ ]:
model_rf = RandomForestRegressor(n_estimators=300, 
                                 random_state=616, 
                                 max_depth=4, 
                                 verbose=0)

model_rf.fit(X, Y)

Y_pred = model_rf.predict(X)

# Evaluate the model
rmse = mean_squared_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")

## Cross-Validation -- Hyperparameter Tuning with GridSearchCV

In this cell, we define a `RandomForestRegressor` model with the `absolute_error` criterion and use `GridSearchCV` to find the best hyperparameters.  
We specify a range of values for `max_depth`, `min_samples_leaf`, `min_samples_split`, `max_features`, and `n_estimators`.  
`GridSearchCV` performs a 5-fold cross-validation (`cv=5`) to evaluate different parameter combinations, using parallel computation (`n_jobs=-2`) for efficiency.


In [ ]:
rf = RandomForestRegressor(criterion="absolute_error", random_state=0)

cv_params = {'max_depth': [4,6,8,10], 
             'min_samples_leaf': [2,5],
             'min_samples_split': [5, 10],
             'max_features': [5, 10],
             'n_estimators': [100, 200, 400, 800]
             }  

rf_cv = GridSearchCV(rf, cv_params, cv=5, verbose=1, n_jobs=-2)

## Fit the model - takes ~5 minutes

In [ ]:
%%time

rf_cv.fit(X, Y)
rf_cv.best_params_, rf_cv.best_score_

## Model Evaluation

We now select the best model and train again with the whole dataset. 

In [ ]:
model_rf = RandomForestRegressor(n_estimators=rf_cv.best_params_["n_estimators"], 
                                 random_state=616, 
                                 max_depth=rf_cv.best_params_["max_depth"], 
                                 min_samples_leaf=rf_cv.best_params_["min_samples_leaf"],
                                 min_samples_split=rf_cv.best_params_["min_samples_split"],
                                 criterion="absolute_error",
                                 verbose=0)

model_rf.fit(X, Y)

Y_pred = model_rf.predict(X)

# Evaluate the model
rmse = mean_squared_error(Y, Y_pred)
r2 = r2_score(Y, Y_pred)

print(f"RMSE: {rmse}")
print(f"R² Score: {r2}")                    

We can visualize the quality of the fit. 

In [ ]:
xline = np.linspace(0.0, 100, 100)

fig, ax = plt.subplots(figsize=(10, 6))
plt.scatter(Y, Y_pred, color='blue', alpha=0.4, s=50)
plt.plot(xline, xline, color='red', linestyle='--')

plt.title(f'Cross-Validation Predictions vs Actual')
plt.xlabel(r'Measured $X_{fv}$', fontsize=16)
plt.ylabel(r'Predicted $X_{fv}$', fontsize=16)

plt.xlim(0,100)
plt.ylim(0,100)
# plt.axes().set_aspect('equal')
ax.set_aspect('equal')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.title("Random Forest Prediction", fontsize=16)

plt.grid(True)
plt.tight_layout()
# plt.savefig("figures/regression_plot.pdf", format="pdf")

plt.show()

In [ ]:
importances = model_rf.feature_importances_
feature_names = X.columns

## Feature Importance Analysis

We extract and analyze the feature importances from the trained `RandomForestRegressor` model.  We use 3 alternative feature importance methods.
 
We store this information in a DataFrame, sort it in descending order, and print the results to identify the most influential features.

### I - Gini Index
The `feature_importances_` attribute provides the Gini importance scores, which indicate how much each feature contributes to the model's predictions. 

In [ ]:
importances = model_rf.feature_importances_
feature_imp_gini = pd.DataFrame({'Feature': feature_names, 'Gini Importance': importances}).sort_values('Gini Importance', ascending=False) 
print(feature_imp_gini.head(10))

In [ ]:
fig = plt.figure(figsize=(18, 10))


plt.barh(feature_imp_gini["Feature"].head(50), feature_imp_gini["Gini Importance"].head(50), color='skyblue')
plt.xlabel('Gini Importance')
plt.title('Feature Importance - Gini Importance')
plt.gca().invert_yaxis()  # Invert y-axis for better visualization
plt.show()

In [ ]:
feature_imp_gini["Order Gini"] = np.arange(1, feature_imp_gini.shape[0]+1)
feature_imp_gini.head(5)

### II - Permutation Test

we compute permutation importance for the trained `RandomForestRegressor` model.  
Permutation importance measures how much the model’s performance decreases when a feature’s values are randomly shuffled, providing an alternative view of feature importance.  
We use the `permutation_importance` function with 10 repeats (`n_repeats=10`).

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(model_rf, X, Y, n_repeats=10, random_state=0, n_jobs=-1)

feature_imp_perm = pd.DataFrame({'Feature': feature_names, 'Permutation Importance': result.importances_mean}).sort_values('Permutation Importance', ascending=False)
# print(feature_imp_perm)

feature_imp_perm["Order Perm"] = np.arange(1, feature_imp_perm.shape[0]+1)
feature_imp_perm.head(7)

In [ ]:
fig = plt.figure(figsize=(10, 12))

plt.barh(feature_imp_perm["Feature"], feature_imp_perm["Permutation Importance"], color='skyblue')
plt.xlabel('Gini Importance')
plt.title('Feature Importance - Permutation Importance')
plt.gca().invert_yaxis()  # Invert y-axis for better visualization
plt.show()

### III - Shapely values

This is a much better method, but we will see it gives very similar results. 

In [ ]:
explainer = shap.TreeExplainer(model_rf)

In [ ]:
# Calculate Shap values
choosen_instance = X.loc[[21]]
shap_values = explainer.shap_values(choosen_instance)
shap.initjs()
shap.force_plot(explainer.expected_value[0], shap_values[0], choosen_instance)

Rather than use a typical feature importance bar chart, we use a density scatter plot of SHAP values for each feature to identify how much impact each feature has on the model output for individuals in the validation dataset. 

In [ ]:
choosen_instance = X
shap_values = explainer.shap_values(choosen_instance)
shap.summary_plot(shap_values, X)

In [ ]:
feature_order = np.array(X.columns[np.argsort(np.abs(shap_values).mean(0))])[::-1]
feature_shap = np.sort(np.abs(shap_values).mean(0))[::-1]
feature_imp_shap = pd.DataFrame({"Feature": feature_order,
                                  "Shap Value": feature_shap,
                                  "Order Shap": np.arange(1, X.shape[1]+1)})
feature_imp_shap.head()

# Summary

## Figures in the paper

In [ ]:
df_summary = pd.merge(feature_imp_gini, feature_imp_perm, how="inner", on="Feature")
df_summary = pd.merge(df_summary, feature_imp_shap, how="inner", on="Feature")
df_summary["Order total"] = df_summary["Order Gini"] + df_summary["Order Perm"] + df_summary["Order Shap"]
df_summary = df_summary.sort_values(by = "Order total")

In [ ]:
df_short = df_summary[["Feature", "Order Gini", "Order Perm", "Order Shap"]].set_index("Feature").head(10)
df_short = df_short.rename(columns={"Order Gini": "Gini Index", "Order Perm": "Permutation Index", "Order Shap": "SHAP Value"})

In [ ]:
print(df_short)

In [ ]:
df_short
ax = sns.heatmap(df_short.transpose(), 
                 # annot=True, 
                 linewidth=4.0, 
                 cmap=sns.color_palette("Spectral", 9), 
                 square=True, cbar=False, annot_kws={"size":14})

ax.set_title("Feature Importance Rankings", fontsize=16)
ax.set_xlabel("Component", fontsize=14)
# ax.set_ylabel("Criteria", fontsize=14)

ax.set_yticklabels(["Gini\nIndex", "Permutation\nIndex", "SHAP\nValue"], rotation=0)

plt.tight_layout()
# plt.savefig("figures/rankings.pdf", format="pdf")

In [ ]:
df_short.transpose()

In [ ]:
df_values = df_summary[["Feature", "Gini Importance", "Permutation Importance", "Shap Value"]].set_index("Feature")
df_values["Gini Importance (norm)"] = df_values["Gini Importance"] / np.sum(df_values["Gini Importance"])
df_values["Permutation Importance (norm)"] = df_values["Permutation Importance"] / np.sum(df_values["Permutation Importance"])
df_values["Shap Value (norm)"] = df_values["Shap Value"] / np.sum(df_values["Shap Value"])

df_values.head(5)

In [ ]:
df_values.index

In [ ]:
fig, ax_ranking = plt.subplots(figsize=(7,10))

plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

plt.grid(axis = 'x', linestyle = '--', linewidth = 0.5)
plt.grid(axis = 'y')

plt.scatter(df_values["Gini Importance (norm)"], np.arange(df_values.shape[0]+1, 1, -1), label="Gini Importance", marker="D", c="#c0392b", s=50, zorder = 100)
plt.scatter(df_values["Permutation Importance (norm)"], np.arange(df_values.shape[0]+1, 1, -1), label="Permutation Importance", marker="o", c="#27ae60", s=50, zorder = 100)
plt.scatter(df_values["Shap Value (norm)"], np.arange(df_values.shape[0]+1, 1, -1), label="Shap Value", marker="s", c="#8e44ad", s=50, zorder = 100)

plt.xlabel("Normalized index", fontsize=16)
plt.ylabel("Components", fontsize=16)
ax_ranking.set_yticks(np.arange(df_values.shape[0]+1, 1, -1), df_values.index)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.xlim(0,0.18)
plt.ylim(28.4, 53.5)
plt.title("Feature Importance Ranking", fontsize=16)

leg = plt.legend(title="Feature Importance Index", loc="lower right", fontsize=16, title_fontsize=16)

# plt.savefig("figures/importance.pdf", format="pdf")